In [3]:
%cd data

/home/dockeruser/data


In [17]:
%ls

anthony.csv*  denarcus.csv*  harden.csv*  lebron.csv*  oneal.csv*
curry.csv*    duncan.csv*    jordan.csv*  magic.csv*   pippen.csv*


# 1. 분석의 목적

## 각 경기별 선수 데이터를 트레이닝, 추가되는 경기 데이터를 바탕으로 classification 테스트. <br />목적은 뻘짓이지만 구분이 된다는 가정하에 분석을 진행

## 구분이 정확해지면 모형 내부 분석 기준을 바탕으로 선수 플레이 스타일을 구분할 수 있으리라 판단

# 2. 데이터 수집

## 1. PG, SG, SF, PF, C 각 포지션 별 역대 BEST 선수와 현역 BEST (주관적임) 선정
### (테스트용으로 구분이 명확하게 될 수 있는 선수로 우선 선정, 차후 빌드업 시 선수 샘플 추가)

PG1 = 매직존슨<br />
PG2 = 스테판커리<br />
SG1 = 마이클조던<br />
SG2 = 제임스하든<br />
SF1 = 스코티피펜(주관적임)<br />
SF1 = 르브론제임스<br />
PF1 = 팀던컨<br />
PF1 = 드마커스(잘 모르겠음)<br />
C1 = 샤킬오닐<br />
C1 = 안소니 데이비스(주관적임)<br />

## 2. 샘플 선수의 5시즌 연속 전체 경기 데이터 수집<br />

## 플레이어 데이터 로딩

In [9]:
pg1 = pd.read_csv('magic.csv')

In [10]:
pg2 = pd.read_csv('curry.csv')

In [11]:
sg1 = pd.read_csv('jordan.csv')

In [12]:
sg2 = pd.read_csv('harden.csv')

In [13]:
sf1 = pd.read_csv('pippen.csv')

In [14]:
sf2 = pd.read_csv('lebron.csv')

In [15]:
pf1 = pd.read_csv('duncan.csv')

In [18]:
pf2 = pd.read_csv('demarcus.csv')

In [20]:
c1 = pd.read_csv('oneal.csv')

In [21]:
c2 = pd.read_csv('anthony.csv')

## 데이터 확인

In [23]:
len(pg1.columns)

20

In [35]:
pd.options.display.max_rows = 400

In [63]:
pg1.tail()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
368,37:00,8,14,0.571,0,0,NaN,8,9,0.889,...,7,7,7,2,0,5,4,24,19.4,0
369,40:00,6,15,0.400,4,8,0.500,8,8,1.000,...,13,17,17,3,1,5,3,24,32.0,0
370,41:00,6,15,0.400,1,4,0.250,6,6,1.000,...,6,6,16,1,1,4,4,19,20.0,0
371,38:00,9,19,0.474,2,7,0.286,3,4,0.750,...,7,7,10,2,0,4,2,23,19.2,0
372,40:00,8,14,0.571,4,4,1.000,9,9,1.000,...,7,9,21,1,0,2,2,29,38.8,0


## 변수설명

MP: 플레이 시간(분)<br />
FG: 필드골<br />
FGA: 필드골 시도<br />
FG%: 필드골 성공률<br />
3P: 3점 슛<br />
3PA: 3점 슛 시도<br />
3P%: 3점 슛 성공률<br />
FT: 프리드로우<br />
FTA: 프리드로우 시도<br />
FT%: 프리드로우 성공률<br />
ORB: 공격 리바운드<br />
DRB: 수비 리바운드<br />
TRB: 총 리바운드<br />
AST: 어시스트<br />
BLK: 블록샷<br />
TOV: 턴오버<br />
PF: 파울<br />
PTS: 득점<br />
GmSc: 게임스코어 (2차 스탯 같은데 의미를 정확하게 모르겠음)<br />

## 라벨링

In [60]:
pg1['y'] = 0
pg2['y'] = 1
sg1['y'] = 2
sg2['y'] = 3
sf1['y'] = 4
sf2['y'] = 5
pf1['y'] = 6
pf2['y'] = 7
c1['y'] = 8
c2['y'] = 9

In [61]:
pg2.head()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,30:22,2,14,0.143,0,6,0.000,1,3,0.333,...,4,7,3,2,0,4,3,5,-2.6,1
1,42:30,10,20,0.500,6,10,0.600,0,1,0.000,...,4,5,7,1,0,4,3,26,18.2,1
2,43:02,6,16,0.375,2,7,0.286,9,10,0.900,...,4,4,4,3,0,5,4,23,14.2,1
3,39:02,3,15,0.200,2,6,0.333,4,5,0.800,...,5,6,8,1,0,3,2,12,7.3,1
4,36:28,8,14,0.571,2,6,0.333,3,3,1.000,...,5,5,6,2,0,3,3,21,17.9,1


In [62]:
c2.head()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,29:04,6,12,0.500,0,0,NaN,9,9,1.000,...,4,7,0,1,1,0,0,21,20.0,9
1,14:25,4,6,0.667,0,0,NaN,0,0,NaN,...,4,6,0,1,2,1,0,8,9.4,9
2,36:30,9,18,0.500,0,1,0.0,5,9,0.556,...,7,11,2,2,5,1,2,23,22.4,9
3,29:03,2,7,0.286,0,0,NaN,4,4,1.000,...,2,4,1,2,3,1,4,8,8.1,9
4,29:08,4,14,0.286,0,1,0.0,0,0,NaN,...,6,11,1,0,2,2,1,8,4.8,9


## 데이터 합치기

In [103]:
df = pd.DataFrame(np.vstack([pg1, pg2, sg1, sg2, sf1, sf2, pf1, pf2, c1, c2]),
                 columns=['MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', 'y'])

In [104]:
df.head()

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,38:00,5,11,0.455,0,0,NaN,6,7,0.857,...,5,5,11,1,1,6,3,16,13.6,0
1,40:00,4,7,0.571,0,0,NaN,9,11,0.818,...,8,9,14,2,1,4,3,17,23.3,0
2,39:00,4,8,0.5,1,1,1,3,4,0.75,...,9,11,13,4,1,8,1,12,17.1,0
3,38:00,5,9,0.556,0,0,NaN,0,0,NaN,...,4,6,14,1,0,4,3,10,13.9,0
4,43:00,9,19,0.474,0,0,NaN,11,11,1,...,6,12,14,1,0,2,5,29,32.1,0


In [75]:
len(df)

3716

## NaN값 확인 및 처리

FGA, FG 가 모두 0 일 경우 FG% 가 NaN<br />
3PA, 3P 가 모두 0 일 경우 3P% 가 NaN<br /><br />

시도하지 않은 투구에 대해 0 퍼센트의 성공률이라고 하기는 힘들지만 편의상 0으로 간주

In [56]:
df.fillna(0)

,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,5,11,0.455,0,0,0.000,6,7,0.857,0,5,5,11,1,1,6,3,16,13.6,0
1,4,7,0.571,0,0,0.000,9,11,0.818,1,8,9,14,2,1,4,3,17,23.3,0
2,4,8,0.500,1,1,1.000,3,4,0.750,2,9,11,13,4,1,8,1,12,17.1,0
3,5,9,0.556,0,0,0.000,0,0,0.000,2,4,6,14,1,0,4,3,10,13.9,0
4,9,19,0.474,0,0,0.000,11,11,1.000,6,6,12,14,1,0,2,5,29,32.1,0
5,8,9,0.889,0,0,0.000,6,7,0.857,0,5,5,13,2,0,2,4,22,27.5,0
6,7,10,0.700,0,0,0.000,6,8,0.750,2,6,8,15,2,0,3,4,20,26.1,0
7,13,17,0.765,0,0,0.000,4,6,0.667,5,8,13,16,3,0,4,2,30,37.8,0
8,3,3,1.000,0,0,0.000,0,0,0.000,1,5,6,17,4,1,2,0,6,21.9,0
9,8,13,0.615,0,1,0.000,7,10,0.700,1,8,9,22,3,1,2,1,23,35.7,0


### 빈 곳은 다 채웠음 이제부터 데이터 처리 그 전에 세이브 한번

In [26]:
df.to_csv('playerclassification.csv')

### 데이터전처리 시작

In [50]:
df = pd.read_csv('playerclassification.csv')

In [61]:
df.head()

,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,5,11,0.455,0,0,NaN,6,7,0.857,0,5,5,11,1,1,6,3,16,13.6,0
1,4,7,0.571,0,0,NaN,9,11,0.818,1,8,9,14,2,1,4,3,17,23.3,0
2,4,8,0.500,1,1,1.0,3,4,0.750,2,9,11,13,4,1,8,1,12,17.1,0
3,5,9,0.556,0,0,NaN,0,0,NaN,2,4,6,14,1,0,4,3,10,13.9,0
4,9,19,0.474,0,0,NaN,11,11,1.000,6,6,12,14,1,0,2,5,29,32.1,0


In [62]:
df = df.drop('FG%', axis=1)

In [63]:
df = df.drop('3P%', axis=1)

In [64]:
df = df.drop('FT%', axis=1)

In [65]:
df.head()

,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,5,11,0,0,6,7,0,5,5,11,1,1,6,3,16,13.6,0
1,4,7,0,0,9,11,1,8,9,14,2,1,4,3,17,23.3,0
2,4,8,1,1,3,4,2,9,11,13,4,1,8,1,12,17.1,0
3,5,9,0,0,0,0,2,4,6,14,1,0,4,3,10,13.9,0
4,9,19,0,0,11,11,6,6,12,14,1,0,2,5,29,32.1,0


## 스케일링

In [66]:
from sklearn.preprocessing import scale

In [69]:
df = pd.DataFrame(StandardScaler().fit_transform(df),
                  columns =['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', 'y'])

In [70]:
df.head()

,FG,FGA,3P,3PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,y
0,-1.178558,-1.246351,-0.644803,-0.838558,0.025989,-0.167636,-1.090180,-0.396645,-0.775302,1.254442,-0.439274,-0.154437,1.402082,0.235390,-1.036257,-0.902200,-1.534035
1,-1.463165,-1.946334,-0.644803,-0.838558,0.817583,0.700263,-0.541026,0.495747,0.159185,2.006621,0.240019,-0.154437,0.375049,0.235390,-0.925013,0.249455,-1.534035
2,-1.463165,-1.771338,-0.074984,-0.562211,-0.765605,-0.818560,0.008128,0.793211,0.626428,1.755895,1.598605,-0.154437,2.429115,-1.133484,-1.481236,-0.486654,-1.534035
3,-1.178558,-1.596343,-0.644803,-0.838558,-1.557199,-1.686458,0.008128,-0.694109,-0.541680,2.006621,-0.439274,-0.832790,0.375049,0.235390,-1.703725,-0.866582,-1.534035
4,-0.040133,0.153616,-0.644803,-0.838558,1.345312,0.700263,2.204743,-0.099181,0.860049,2.006621,-0.439274,-0.832790,-0.651983,1.604265,0.409923,1.294256,-1.534035


In [71]:
df.to_csv('playerclassification.csv')